In [2]:
from src.code.tools.metrics import calculate_metrics
import os
from src.code.TFIDF.pre_processing import Processing
from src.code.bert.bert import Bert
from dotenv import load_dotenv
from tqdm import tqdm
import glob
from nltk.tokenize import sent_tokenize
load_dotenv()

True

In [ ]:
metricis = {'Précision': 0,
            'Rappel': 0,
            'F-mesure': 0,
            'Spécificité': 0,
            'Taux de faux positifs': 0}
documents_count = 0
for file in tqdm(glob.glob(os.path.join(os.getenv('TXT_FOLDER2'), "*.txt"))[1:2]):
    with open(file, "r", encoding="utf-8") as f:
        doc = f.read().strip().lower()
        if len(doc) > 10:
            documents_count += 1
            documents = sent_tokenize(doc)
            bert = Bert(documents=documents)
            bert.processing()
            top_doc_indices_bert, resume_doc_bert = bert.get_resume(top_n=1)

            preprocessing = Processing(
                log_file="", corpus_dir="processed", documents=documents)
            preprocessing.process_corpus()
            top_doc_indices_keywords, resume_doc_keywords = preprocessing.get_resume_docs(
                top_n=1, number_key_words=30)
            P_bert, R_bert, F1_bert = bert.get_bert_score(
                resume_doc_bert[0], doc)
            P_keyword, R_keyword, F1_key_word = bert.get_bert_score(
                resume_doc_keywords[0], doc)
            print(f"Précision des deux méthode vaut; Méthode de Keyword: {
                  P_keyword}, Méthode Bert: {P_bert}")
            print(f"Le Rappel des deux méthode vaut; Méthode Keyword {
                  R_keyword}, La méthode Bert : {R_bert}")
            print(f"La F-mesure des deux méthode vaut; Méthode Keyword: {
                  F1_key_word}, Méthode Bert: {F1_bert}")

100%|██████████| 784/784 [00:00<00:00, 853447.79it/s]


ai​ ​now​ ​2017​ ​report  authors   alex​ ​campolo, ​ ​new​ ​york​ ​university  madelyn​ ​sanfilippo, ​ ​new​ ​york​ ​university  meredith​ ​whittaker, ​ ​google​ ​open​ ​research,​ ​new​ ​york​ ​university, ​ ​and​ ​ai​ ​now  ​ ​kate ​ ​crawford, ​ ​microsoft​ ​research,​ ​new​ ​york​ ​university, ​ ​and​ ​ai​ ​now  editors   andrew​ ​selbst,​ ​yale​ ​information​ ​society​ ​project​ ​and​ ​data​ ​&​ ​society  solon​ ​barocas, ​ ​cornell​ ​university  table​ ​of​ ​contents   recommendations 1  executive​ ​summary 3  introduction 6  labor​ ​and​ ​automation 7  research ​ ​by​ ​sector​ ​and​ ​task 7  ai​ ​and​ ​the​ ​nature​ ​of​ ​work 9  inequality​ ​and​ ​redistribution 1​3  bias​ ​and​ ​inclusion 1​3  where​ ​bias​ ​comes ​ ​from 1​4  the​ ​ai​ ​field​ ​is​ ​not​ ​diverse 1​6  recent​ ​developments ​ ​in​ ​bias​ ​research 1​8  emerging ​ ​strategies​ ​to​ ​address​ ​bias 20  rights​ ​and​ ​liberties 21  population ​ ​registries ​ ​and​ ​computing​ ​power 2​2  corporate ​ ​and​ ​gover

100%|██████████| 1/1 [00:27<00:00, 27.31s/it]


In [ ]:
      """ metricis_calcul = calculate_metrics(top_indices, resume_indeces)
            metricis['Précision'] += metricis_calcul['Précision']
            metricis['Rappel'] += metricis_calcul['Rappel']
            metricis['F-mesure'] += metricis_calcul['F-mesure']
            metricis['Spécificité'] += metricis_calcul['Spécificité'] """

            """  P_keyword, R_keyword, F1_key_word = bert.get_bert_score(
                resume_doc_keywords[0], doc)
            P_bert, R_bert, F1_bert = bert.get_score(resume_doc_bert[0], doc)
            print(f"Précision des deux méthode vaut; Méthode de Keyword: {
                  P_keyword}, Méthode Bert: {P_bert}")
            print(f"Le Rappel des deux méthode vaut; Méthode Keyword {
                  R_keyword}, La méthode Bert : {R_bert}")
            print(f"La F-mesure des deux méthode vaut; Méthode Keyword: {
                  F1_key_word}, Méthode Bert: {F1_bert}") """


""" metricis['Précision'] /= documents_count
metricis['Rappel'] /= documents_count
metricis['F-mesure'] /= documents_count
metricis['Spécificité'] /= documents_count

print(metricis) """